In [1]:
import sys
import time
import requests
import random
import pandas as pd
from datetime import date, datetime, timedelta
import traceback
import csv
import math
import time

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 1000)

In [2]:
file = '/Users/keriwheatley/Desktop/random_code/googlemaps_web_scraper/cities.csv'
cities = pd.read_csv(file, encoding = "ISO-8859-1")
keyword = 'metrology'

api_key = ''

In [68]:
# for i, city in cities.head(10).iterrows():
#     print(city['CITY'])

In [3]:
cities = cities.head(2)

In [4]:
# cities = cities[cities['CITY'] != 'New York New York']

In [5]:
data = pd.DataFrame(columns=['place_id','name','formatted_address','rating','user_ratings_total','types','geometry'])
print('Start:',datetime.now())
for index, city in cities.iterrows():
    print(city['CITY'])
    query=keyword+' in '+city['CITY']
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
    time.sleep(5)
    r = requests.get(url + '&query=' + query +
                            '&key=' + api_key) 
    x = r.json()['results']
    
    for i in range(len(x)): 
        place_id = x[i]['place_id']
        formatted_address = x[i]['formatted_address']
        name = x[i]['name']
        rating = x[i]['rating']
        user_ratings_total = x[i]['user_ratings_total']
        types = x[i]['types']
        geometry = x[i]['geometry']['location']
        current_row = pd.DataFrame([[place_id,name,formatted_address,rating,user_ratings_total,types,geometry]]
                                   ,columns=['place_id','name','formatted_address','rating','user_ratings_total','types','geometry'])
        data = pd.concat([data,current_row], sort=False)
print('End:',datetime.now())

Start: 2019-05-01 16:04:53.882798
New York New York
Los Angeles California
End: 2019-05-01 16:05:04.838754


In [71]:
# data.count()

In [6]:
# data.to_csv(keyword.replace(' ','_')+'data.csv')
data

,place_id,name,formatted_address,rating,user_ratings_total,types,geometry


In [73]:
# list_places


In [75]:
list_places = data['place_id']
# list_places = ['ChIJp9LTWDWej4ARrh0N-FMPgso']
# list_places = data[~data['place_id'].isin(data2['place_id'])]['place_id']

data2 = pd.DataFrame(columns=['place_id','formatted_phone_number','website','opening_hours','name','adr_address'])

print('Start:',datetime.now())
for place_id in list_places:
#     print(place_id)
    url = "https://maps.googleapis.com/maps/api/place/details/json?placeid="+place_id+"&fields=adr_address,name,website,formatted_phone_number,opening_hours"
    r = requests.get(url + '&key=' + api_key) 
    x = r.json()['result']

    if 'formatted_phone_number' in x.keys():
        formatted_phone_number = x['formatted_phone_number']
    else:
        formatted_phone_number = ''

    if 'name' in x.keys():
        name = x['name']
    else:
        name = ''

    if 'website' in x.keys():
        website = x['website']
    else:
        website = ''

    if 'opening_hours' in x.keys():
        opening_hours = x['opening_hours']
    else:
        opening_hours = ''

    if 'adr_address' in x.keys():
        adr_address = x['adr_address']
    else:
        adr_address = ''
        
    current_row = pd.DataFrame([[place_id,formatted_phone_number,website,opening_hours,name,adr_address]]
                               ,columns=['place_id','formatted_phone_number','website','opening_hours','name','adr_address'])
    data2 = pd.concat([data2,current_row], sort=False) 
print('End:',datetime.now())    

Start: 2019-04-29 18:55:17.270550
End: 2019-04-29 18:56:59.565032


In [76]:
# data2
# current_row
# data[data['place_id']=='ChIJp9LTWDWej4ARrh0N-FMPgso']
# data[~data['place_id'].isin(data2['place_id'])]

In [77]:
data2.to_csv(keyword.replace(' ','_')+'data2.csv')

In [78]:
final_data = data.merge(data2, left_on=['place_id','name'], right_on=['place_id','name'], how='outer')

In [79]:
# for column in final_data.columns:
#     print(column)
#     final_data[column] = final_data[column].astype(str)

In [80]:
for column in final_data.columns:
    final_data[column] = final_data[column].astype(str)
final_data = final_data.drop_duplicates()

In [81]:
final_data.to_csv(keyword.replace(' ','_')+'places_list.csv', index=False)

In [82]:
final_data.count()

place_id                  381
name                      381
formatted_address         381
rating                    381
user_ratings_total        381
types                     381
geometry                  381
formatted_phone_number    381
website                   381
opening_hours             381
adr_address               381
dtype: int64